<h1 class='font-effect-3d' style='font-family:Ewert; color:#ff355e'>Modules, Helpful Functions, & Styling</h1>

In [1]:
!pip install --upgrade neural_structured_learning --user

     |████████████████████████████████| 104 kB 3.3 MB/s 


In [2]:
import warnings; warnings.filterwarnings('ignore')
import h5py,pylab as pl,pandas as pd
import numpy as np,tensorflow as tf
import neural_structured_learning as nsl
from sklearn.model_selection import train_test_split

In [3]:
%%html
<style>
@import url('https://fonts.googleapis.com/css?family=Ewert|Roboto&effect=3d');
span {font-family:Roboto; color:black; text-shadow:3px 3px 3px #aaa;}  
div.output_area pre{font-family:Roboto; font-size:110%; color:#ff603b;}      
</style>

In [4]:
fpath3='../input/classification-of-handwritten-letters/'
fpath4='../input/flower-color-images/'
def prepro(x_train,y_train,x_test,y_test,n_class):
    n=int(len(x_test)/2)    
    x_valid,y_valid=x_test[:n],y_test[:n]
    x_test,y_test=x_test[n:],y_test[n:]
    cy_train=tf.keras.utils.to_categorical(y_train,n_class) 
    cy_valid=tf.keras.utils.to_categorical(y_valid,n_class)
    cy_test=tf.keras.utils.to_categorical(y_test,n_class)
    df=pd.DataFrame([[x_train.shape,x_valid.shape,x_test.shape],
                     [y_train.shape,y_valid.shape,y_test.shape],
                     [cy_train.shape,cy_valid.shape,cy_test.shape]],
                    columns=['train','valid','test'],
                    index=['images','labels','encoded labels'])
    display(df)
    return [[x_train,x_valid,x_test],
            [y_train,y_valid,y_test],
            [cy_train,cy_valid,cy_test]]

<h1 class='font-effect-3d' style='font-family:Ewert; color:#ff355e'>Data Loading & Preprocessing</h1>

In [5]:
(x_train1,y_train1),(x_test1,y_test1)=\
tf.keras.datasets.mnist.load_data()
[[x_train1,x_valid1,x_test1],
 [y_train1,y_valid1,y_test1],
 [cy_train1,cy_valid1,cy_test1]]=\
prepro(x_train1/255,y_train1.reshape(-1,1),
       x_test1/255,y_test1.reshape(-1,1),10)

11493376/11490434 [==============================] - 0s 0us/step


,train,valid,test
images,"(60000, 28, 28)","(5000, 28, 28)","(5000, 28, 28)"
labels,"(60000, 1)","(5000, 1)","(5000, 1)"
encoded labels,"(60000, 10)","(5000, 10)","(5000, 10)"


In [6]:
(x_train2,y_train2),(x_test2,y_test2)=\
tf.keras.datasets.cifar10.load_data()
[[x_train2,x_valid2,x_test2],
 [y_train2,y_valid2,y_test2],
 [cy_train2,cy_valid2,cy_test2]]=\
prepro(x_train2/255,y_train2,x_test2/255,y_test2,10)

170500096/170498071 [==============================] - 3s 0us/step


,train,valid,test
images,"(50000, 32, 32, 3)","(5000, 32, 32, 3)","(5000, 32, 32, 3)"
labels,"(50000, 1)","(5000, 1)","(5000, 1)"
encoded labels,"(50000, 10)","(5000, 10)","(5000, 10)"


In [7]:
f=h5py.File(fpath3+'LetterColorImages_123.h5','r') 
keys=list(f.keys()); keys
images=np.array(f[keys[1]])/255
labels=np.array(f[keys[2]]).astype('int').reshape(-1,1)-1
x_train3,x_test3,y_train3,y_test3=\
train_test_split(images,labels,test_size=.2,random_state=1)
del images,labels
[[x_train3,x_valid3,x_test3],
 [y_train3,y_valid3,y_test3],
 [cy_train3,cy_valid3,cy_test3]]=\
prepro(x_train3,y_train3,x_test3,y_test3,33)

,train,valid,test
images,"(11352, 32, 32, 3)","(1419, 32, 32, 3)","(1419, 32, 32, 3)"
labels,"(11352, 1)","(1419, 1)","(1419, 1)"
encoded labels,"(11352, 33)","(1419, 33)","(1419, 33)"


In [8]:
f=h5py.File(fpath4+'FlowerColorImages.h5','r') 
keys=list(f.keys()); keys
images=np.array(f[keys[0]])/255
labels=np.array(f[keys[1]]).astype('int').reshape(-1,1)
x_train4,x_test4,y_train4,y_test4=\
train_test_split(images,labels,test_size=.2,random_state=1)
del images,labels
[[x_train4,x_valid4,x_test4],
 [y_train4,y_valid4,y_test4],
 [cy_train4,cy_valid4,cy_test4]]=\
prepro(x_train4,y_train4,x_test4,y_test4,10)

,train,valid,test
images,"(168, 128, 128, 3)","(21, 128, 128, 3)","(21, 128, 128, 3)"
labels,"(168, 1)","(21, 1)","(21, 1)"
encoded labels,"(168, 10)","(21, 10)","(21, 10)"


<h1 class='font-effect-3d' style='font-family:Ewert; color:#ff355e'>Models with Adversarial Regularization</h1>
MLP Base

In [9]:
batch_size=64; img_size=28; n_class=10; epochs=7
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size),name='input'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation=tf.nn.relu),
    tf.keras.layers.BatchNormalization(),    
    tf.keras.layers.Dense(256,activation=tf.nn.relu),
    tf.keras.layers.Dense(n_class,activation=tf.nn.softmax)
])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [10]:
train1=tf.data.Dataset.from_tensor_slices(
    {'input':x_train1,'label':y_train1}).batch(batch_size)
valid1=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid1,'label':y_valid1}).batch(batch_size)
valid_steps=x_valid1.shape[0]//batch_size
adv_model.fit(train1,validation_data=valid1,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Train for 938 steps, validate for 78 steps
Epoch 1/7
938/938 - 8s - loss: 0.2686 - sparse_categorical_crossentropy: 0.2163 - sparse_categorical_accuracy: 0.9347 - adversarial_loss: 0.2612 - val_loss: 0.2035 - val_sparse_categorical_crossentropy: 0.1640 - val_sparse_categorical_accuracy: 0.9489 - val_adversarial_loss: 0.1978
Epoch 2/7
938/938 - 5s - loss: 0.1158 - sparse_categorical_crossentropy: 0.0913 - sparse_categorical_accuracy: 0.9718 - adversarial_loss: 0.1223 - val_loss: 0.1851 - val_sparse_categorical_crossentropy: 0.1488 - val_sparse_categorical_accuracy: 0.9531 - val_adversarial_loss: 0.1819
Epoch 3/7
938/938 - 5s - loss: 0.0765 - sparse_categorical_crossentropy: 0.0591 - sparse_categorical_accuracy: 0.9818 - adversarial_loss: 0.0868 - val_loss: 0.1480 - val_sparse_categorical_crossentropy: 0.1184 - val_sparse_categorical_accuracy: 0.9661 - val_adversarial_loss: 0.1481
Epoch 4/7
938/938 - 5s - loss: 0.0524 - sparse_categorical_crossentropy: 0.0394 - sparse_categorical_accurac

In [11]:
adv_model.evaluate({'input':x_test1,'label':y_test1})

5000/5000 [==============================] - 1s 178us/sample - loss: 0.0667 - sparse_categorical_crossentropy: 0.0525 - sparse_categorical_accuracy: 0.9856 - adversarial_loss: 0.0694


[0.06670803442965553, 0.052518163, 0.9856, 0.06935637]

In [12]:
batch_size=64; img_size=32; n_class=10; epochs=7
base_model=tf.keras.models.Sequential([
        tf.keras.Input((img_size,img_size,3),name='input'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(256,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.BatchNormalization(),   
        tf.keras.layers.Dense(1024,activation='relu'),
        tf.keras.layers.Dense(10,activation='softmax')
    ])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [13]:
train2=tf.data.Dataset.from_tensor_slices(
    {'input':x_train2,'label':y_train2}).batch(batch_size)
valid2=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid2,'label':y_valid2}).batch(batch_size)
valid_steps=x_valid2.shape[0]//batch_size
adv_model.fit(train2,validation_data=valid2,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Train for 782 steps, validate for 78 steps
Epoch 1/7
782/782 - 10s - loss: 2.1255 - sparse_categorical_crossentropy: 1.7595 - sparse_categorical_accuracy: 0.3839 - adversarial_loss: 1.8299 - val_loss: 2.1850 - val_sparse_categorical_crossentropy: 1.8106 - val_sparse_categorical_accuracy: 0.3720 - val_adversarial_loss: 1.8717
Epoch 2/7
782/782 - 7s - loss: 1.7868 - sparse_categorical_crossentropy: 1.4789 - sparse_categorical_accuracy: 0.4723 - adversarial_loss: 1.5398 - val_loss: 1.8945 - val_sparse_categorical_crossentropy: 1.5674 - val_sparse_categorical_accuracy: 0.4487 - val_adversarial_loss: 1.6354
Epoch 3/7
782/782 - 7s - loss: 1.6439 - sparse_categorical_crossentropy: 1.3585 - sparse_categorical_accuracy: 0.5168 - adversarial_loss: 1.4269 - val_loss: 2.0274 - val_sparse_categorical_crossentropy: 1.6750 - val_sparse_categorical_accuracy: 0.4333 - val_adversarial_loss: 1.7617
Epoch 4/7
782/782 - 7s - loss: 1.5312 - sparse_categorical_crossentropy: 1.2632 - sparse_categorical_accura

In [14]:
adv_model.evaluate({'input':x_test2,'label':y_test2})

5000/5000 [==============================] - 2s 344us/sample - loss: 2.8210 - sparse_categorical_crossentropy: 2.3192 - sparse_categorical_accuracy: 0.3780 - adversarial_loss: 2.4895


[2.820982490158081, 2.3192065, 0.378, 2.4895182]

In [15]:
batch_size=128; img_size=32; n_class=33; epochs=100
base_model=tf.keras.models.Sequential([
        tf.keras.Input((img_size,img_size,3),name='input'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(256,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.BatchNormalization(),   
        tf.keras.layers.Dense(1024,activation='relu'),
        tf.keras.layers.Dense(33,activation='softmax')
    ])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [16]:
train3=tf.data.Dataset.from_tensor_slices(
    {'input':x_train3,'label':y_train3}).batch(batch_size)
valid3=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid3,'label':y_valid3}).batch(batch_size)
valid_steps=x_valid3.shape[0]//batch_size
adv_model.fit(train3,validation_data=valid3,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Train for 89 steps, validate for 11 steps
Epoch 1/100
89/89 - 4s - loss: 3.2544 - sparse_categorical_crossentropy: 2.6475 - sparse_categorical_accuracy: 0.2504 - adversarial_loss: 3.0346 - val_loss: 5.7747 - val_sparse_categorical_crossentropy: 4.7928 - val_sparse_categorical_accuracy: 0.0384 - val_adversarial_loss: 4.9097
Epoch 2/100
89/89 - 1s - loss: 2.4075 - sparse_categorical_crossentropy: 1.9373 - sparse_categorical_accuracy: 0.4288 - adversarial_loss: 2.3513 - val_loss: 8.6697 - val_sparse_categorical_crossentropy: 7.1791 - val_sparse_categorical_accuracy: 0.0625 - val_adversarial_loss: 7.4529
Epoch 3/100
89/89 - 1s - loss: 2.0734 - sparse_categorical_crossentropy: 1.6562 - sparse_categorical_accuracy: 0.5012 - adversarial_loss: 2.0862 - val_loss: 7.3326 - val_sparse_categorical_crossentropy: 6.0520 - val_sparse_categorical_accuracy: 0.0959 - val_adversarial_loss: 6.4032
Epoch 4/100
89/89 - 1s - loss: 1.7565 - sparse_categorical_crossentropy: 1.3945 - sparse_categorical_accuracy

In [17]:
adv_model.evaluate({'input':x_test3,'label':y_test3})

1419/1419 [==============================] - 1s 457us/sample - loss: 2.4534 - sparse_categorical_crossentropy: 1.8803 - sparse_categorical_accuracy: 0.7301 - adversarial_loss: 2.9610


[2.45341463203242, 1.8803138, 0.73009163, 2.961028]

In [18]:
batch_size=16; img_size=128; n_class=10; epochs=100
base_model=tf.keras.models.Sequential([
        tf.keras.Input((img_size,img_size,3),name='input'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(1024,activation='relu'),
        tf.keras.layers.BatchNormalization(),   
        tf.keras.layers.Dense(1024,activation='relu'),
        tf.keras.layers.Dense(10,activation='softmax')
    ])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [19]:
train4=tf.data.Dataset.from_tensor_slices(
    {'input':x_train4,'label':y_train4}).batch(batch_size)
valid4=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid4,'label':y_valid4}).batch(batch_size)
valid_steps=x_valid4.shape[0]//batch_size
adv_model.fit(train4,validation_data=valid4,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Train for 11 steps, validate for 1 steps
Epoch 1/100
11/11 - 3s - loss: 3.1089 - sparse_categorical_crossentropy: 2.5778 - sparse_categorical_accuracy: 0.2798 - adversarial_loss: 2.6703 - val_loss: 13.9963 - val_sparse_categorical_crossentropy: 11.6599 - val_sparse_categorical_accuracy: 0.1250 - val_adversarial_loss: 11.6820
Epoch 2/100
11/11 - 0s - loss: 1.8852 - sparse_categorical_crossentropy: 1.5613 - sparse_categorical_accuracy: 0.5714 - adversarial_loss: 1.5885 - val_loss: 7.1910 - val_sparse_categorical_crossentropy: 5.9901 - val_sparse_categorical_accuracy: 0.1875 - val_adversarial_loss: 6.0048
Epoch 3/100
11/11 - 0s - loss: 0.4392 - sparse_categorical_crossentropy: 0.3632 - sparse_categorical_accuracy: 0.9286 - adversarial_loss: 0.3738 - val_loss: 10.4913 - val_sparse_categorical_crossentropy: 8.7391 - val_sparse_categorical_accuracy: 0.1250 - val_adversarial_loss: 8.7611
Epoch 4/100
11/11 - 0s - loss: 0.0771 - sparse_categorical_crossentropy: 0.0634 - sparse_categorical_accur

In [20]:
adv_model.evaluate({'input':x_test4,'label':y_test4})

21/21 [==============================] - 0s 22ms/sample - loss: 2.9184 - sparse_categorical_crossentropy: 2.4252 - sparse_categorical_accuracy: 0.4762 - adversarial_loss: 2.4661


[2.918431520462036, 2.425217, 0.47619048, 2.466073]

CNN Base

In [21]:
batch_size=128; img_size=28; n_class=10; epochs=7
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size,1),name='input'),
    tf.keras.layers.Conv2D(32,(5,5),padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Conv2D(196,(5,5)),
    tf.keras.layers.Activation('relu'),    
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.GlobalMaxPooling2D(),    
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(10,activation='softmax')
])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [22]:
train1=tf.data.Dataset.from_tensor_slices(
    {'input':x_train1.reshape(x_train1.shape[0],28,28,1),
     'label':y_train1}).batch(batch_size)
valid1=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid1.reshape(x_valid1.shape[0],28,28,1),
     'label':y_valid1}).batch(batch_size)
valid_steps=x_valid1.shape[0]//batch_size
adv_model.fit(train1,validation_data=valid1,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Train for 469 steps, validate for 39 steps
Epoch 1/7
469/469 - 12s - loss: 0.4530 - sparse_categorical_crossentropy: 0.3753 - sparse_categorical_accuracy: 0.8785 - adversarial_loss: 0.3887 - val_loss: 0.1719 - val_sparse_categorical_crossentropy: 0.1407 - val_sparse_categorical_accuracy: 0.9585 - val_adversarial_loss: 0.1561
Epoch 2/7
469/469 - 6s - loss: 0.1102 - sparse_categorical_crossentropy: 0.0904 - sparse_categorical_accuracy: 0.9726 - adversarial_loss: 0.0991 - val_loss: 0.0795 - val_sparse_categorical_crossentropy: 0.0646 - val_sparse_categorical_accuracy: 0.9818 - val_adversarial_loss: 0.0744
Epoch 3/7
469/469 - 6s - loss: 0.0812 - sparse_categorical_crossentropy: 0.0667 - sparse_categorical_accuracy: 0.9798 - adversarial_loss: 0.0727 - val_loss: 0.0643 - val_sparse_categorical_crossentropy: 0.0520 - val_sparse_categorical_accuracy: 0.9850 - val_adversarial_loss: 0.0615
Epoch 4/7
469/469 - 6s - loss: 0.0660 - sparse_categorical_crossentropy: 0.0539 - sparse_categorical_accura

In [23]:
adv_model.evaluate({'input':x_test1.reshape(x_test1.shape[0],28,28,1),
                    'label':y_test1})

5000/5000 [==============================] - 1s 209us/sample - loss: 0.0198 - sparse_categorical_crossentropy: 0.0158 - sparse_categorical_accuracy: 0.9954 - adversarial_loss: 0.0197


[0.01979501006990904, 0.015771808, 0.9954, 0.019694077]

In [24]:
batch_size=64; img_size=32; n_class=10; epochs=20
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size,3),name='input'),
    tf.keras.layers.Conv2D(32,(5,5),padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Conv2D(96,(5,5)),
    tf.keras.layers.Activation('relu'),    
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.GlobalMaxPooling2D(),    
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(10,activation='softmax')
])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [25]:
train2=tf.data.Dataset.from_tensor_slices(
    {'input':x_train2,'label':y_train2}).batch(batch_size)
valid2=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid2,'label':y_valid2}).batch(batch_size)
valid_steps=x_valid2.shape[0]//batch_size
adv_model.fit(train2,validation_data=valid2,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Train for 782 steps, validate for 78 steps
Epoch 1/20
782/782 - 9s - loss: 1.9742 - sparse_categorical_crossentropy: 1.6399 - sparse_categorical_accuracy: 0.3924 - adversarial_loss: 1.6713 - val_loss: 1.7037 - val_sparse_categorical_crossentropy: 1.4091 - val_sparse_categorical_accuracy: 0.5026 - val_adversarial_loss: 1.4731
Epoch 2/20
782/782 - 8s - loss: 1.5793 - sparse_categorical_crossentropy: 1.3087 - sparse_categorical_accuracy: 0.5303 - adversarial_loss: 1.3535 - val_loss: 1.4707 - val_sparse_categorical_crossentropy: 1.2112 - val_sparse_categorical_accuracy: 0.5701 - val_adversarial_loss: 1.2975
Epoch 3/20
782/782 - 8s - loss: 1.4271 - sparse_categorical_crossentropy: 1.1800 - sparse_categorical_accuracy: 0.5814 - adversarial_loss: 1.2354 - val_loss: 1.3247 - val_sparse_categorical_crossentropy: 1.0877 - val_sparse_categorical_accuracy: 0.6366 - val_adversarial_loss: 1.1848
Epoch 4/20
782/782 - 7s - loss: 1.3309 - sparse_categorical_crossentropy: 1.0991 - sparse_categorical_acc

In [26]:
adv_model.evaluate({'input':x_test2,'label':y_test2})

5000/5000 [==============================] - 1s 214us/sample - loss: 1.0782 - sparse_categorical_crossentropy: 0.8655 - sparse_categorical_accuracy: 0.7020 - adversarial_loss: 1.0599


[1.0782131605148315, 0.8654679, 0.702, 1.0599141]

In [27]:
batch_size=128; img_size=32; n_class=33; epochs=150
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size,3),name='input'),
    tf.keras.layers.Conv2D(32,(5,5),padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Conv2D(196,(5,5)),
    tf.keras.layers.Activation('relu'),    
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.GlobalMaxPooling2D(),    
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(33,activation='softmax')
])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [28]:
train3=tf.data.Dataset.from_tensor_slices(
    {'input':x_train3,'label':y_train3}).batch(batch_size)
valid3=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid3,'label':y_valid3}).batch(batch_size)
valid_steps=x_valid3.shape[0]//batch_size
adv_model.fit(train3,validation_data=valid3,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Train for 89 steps, validate for 11 steps
Epoch 1/150
89/89 - 3s - loss: 4.1883 - sparse_categorical_crossentropy: 3.4903 - sparse_categorical_accuracy: 0.0344 - adversarial_loss: 3.4904 - val_loss: 4.1640 - val_sparse_categorical_crossentropy: 3.4693 - val_sparse_categorical_accuracy: 0.0533 - val_adversarial_loss: 3.4735
Epoch 2/150
89/89 - 1s - loss: 4.0786 - sparse_categorical_crossentropy: 3.3969 - sparse_categorical_accuracy: 0.0567 - adversarial_loss: 3.4084 - val_loss: 4.0189 - val_sparse_categorical_crossentropy: 3.3458 - val_sparse_categorical_accuracy: 0.0724 - val_adversarial_loss: 3.3653
Epoch 3/150
89/89 - 1s - loss: 3.9612 - sparse_categorical_crossentropy: 3.2975 - sparse_categorical_accuracy: 0.0706 - adversarial_loss: 3.3181 - val_loss: 3.9330 - val_sparse_categorical_crossentropy: 3.2724 - val_sparse_categorical_accuracy: 0.0732 - val_adversarial_loss: 3.3030
Epoch 4/150
89/89 - 1s - loss: 3.8665 - sparse_categorical_crossentropy: 3.2165 - sparse_categorical_accuracy

In [29]:
adv_model.evaluate({'input':x_test3,'label':y_test3})

1419/1419 [==============================] - 1s 420us/sample - loss: 0.3448 - sparse_categorical_crossentropy: 0.2441 - sparse_categorical_accuracy: 0.9345 - adversarial_loss: 0.4955


[0.3448011935219284, 0.24412942, 0.9344609, 0.49546784]

In [30]:
batch_size=16; img_size=32; n_class=10; epochs=100
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size,3),name='input'),
    tf.keras.layers.Conv2D(32,(5,5),padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Conv2D(196,(5,5)),
    tf.keras.layers.Activation('relu'),    
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.GlobalMaxPooling2D(),    
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(10,activation='softmax')
])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [31]:
train4=tf.data.Dataset.from_tensor_slices(
    {'input':x_train4,'label':y_train4}).batch(batch_size)
valid4=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid4,'label':y_valid4}).batch(batch_size)
valid_steps=x_valid4.shape[0]//batch_size
adv_model.fit(train4,validation_data=valid4,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Train for 11 steps, validate for 1 steps
Epoch 1/100
11/11 - 2s - loss: 2.8011 - sparse_categorical_crossentropy: 2.3372 - sparse_categorical_accuracy: 0.1488 - adversarial_loss: 2.3195 - val_loss: 2.7812 - val_sparse_categorical_crossentropy: 2.3170 - val_sparse_categorical_accuracy: 0.1250 - val_adversarial_loss: 2.3209
Epoch 2/100
11/11 - 0s - loss: 2.6366 - sparse_categorical_crossentropy: 2.1997 - sparse_categorical_accuracy: 0.2143 - adversarial_loss: 2.1863 - val_loss: 2.6835 - val_sparse_categorical_crossentropy: 2.2355 - val_sparse_categorical_accuracy: 0.1875 - val_adversarial_loss: 2.2397
Epoch 3/100
11/11 - 0s - loss: 2.3447 - sparse_categorical_crossentropy: 1.9552 - sparse_categorical_accuracy: 0.3810 - adversarial_loss: 1.9497 - val_loss: 2.5845 - val_sparse_categorical_crossentropy: 2.1524 - val_sparse_categorical_accuracy: 0.1250 - val_adversarial_loss: 2.1607
Epoch 4/100
11/11 - 0s - loss: 2.0521 - sparse_categorical_crossentropy: 1.7121 - sparse_categorical_accuracy:

In [32]:
adv_model.evaluate({'input':x_test4,'label':y_test4})

21/21 [==============================] - 0s 22ms/sample - loss: 0.8453 - sparse_categorical_crossentropy: 0.6939 - sparse_categorical_accuracy: 0.8571 - adversarial_loss: 0.7569


[0.8452944755554199, 0.6939086, 0.85714287, 0.7569294]